# Install
Important `non-built-in` modules to install.

In [1]:
#!pip install deezer-python
#!pip install deezer-oauth-cli

# Imports

In [2]:
import deezer
import requests
import json
import subprocess
import re
from pathlib import Path

# Config

## Config File

Create a file `_config.hide` with the `json` format as below.<br>
<b>`Make sure only you have access to this information!`</b>

```json
{ 
    "properties": {
        "_deezer_user_id": "your-deezer-user-id",
        "_deezer_recsys_playlist_id": "your-recsys-playlist-id",
        "_deezer_app_id": "your-app-id",
        "_deezer_app_name": "your-app-name",
        "_deezer_app_secret": "your-deezer-app-secret-code"
   }
}
```

In [3]:
with open('_config.hide') as _opened_config_file:
    _config_file = json.load(_opened_config_file)["properties"]

## Deezer Sign In
Now you will be redirect to a `Deezer` login page. At this moment the `app token` will be written to `.env` file.<br>
<b>`Make sure only you have access to this information!`</b>

In [4]:
_deezer_user_id            = _config_file["_deezer_user_id"]
_deezer_recsys_playlist_id = _config_file["_deezer_recsys_playlist_id"]

_deezer_app_id             = _config_file["_deezer_app_id"]
_deezer_app_name           = _config_file["_deezer_app_name"]
_deezer_app_secret         = _config_file["_deezer_app_secret"]

p = subprocess.call(
    ["deezer-oauth", _deezer_app_id, _deezer_app_secret],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT
)

## Read Deezer Token from [.env] File
Then the `app token` will be readen from `.env` file and set as `_deezer_app_token` variable.<br>
<b>`Make sure only you have access to this information!`</b>

In [5]:
variable_name = "API_TOKEN"
env_path = Path(".env")

if env_path.exists():
    _env_file_content = env_path.read_text()
    _deezer_app_token = re.findall("(?<=API_TOKEN=).*", _env_file_content)[0]
else:
    raise Exception("Please, run the DEEZER-OAUTH process above!")

# Functions

In [6]:
def _f_deezer_api(
    _p_method: str,
    _p_url:    str
) -> dict:
    if _p_method == "GET":
        return json.loads(requests.get(_p_url).content)
    elif _p_method == "POST":
        return json.loads(requests.post(_p_url).content)
    elif _p_method == "DELETE":
        return json.loads(requests.delete(_p_url).content)

# Data Acquisition

## User Data

In [7]:
_user_data = _f_deezer_api("GET", "https://api.deezer.com/user/{}?access_token={}".format(_deezer_user_id, _deezer_app_token))

print("Welcome, {}!".format(_user_data["name"]))

Welcome, Fernando Gonçalves!


## User History

In [8]:
_user_recents_1 = _f_deezer_api("GET", "https://api.deezer.com/user/{}/history?access_token={}".format(_deezer_user_id, _deezer_app_token))["data"]
_user_recents_2 = _f_deezer_api("GET", "https://api.deezer.com/user/{}/history?access_token={}&index=50".format(_deezer_user_id, _deezer_app_token))["data"]

_user_recent_artists_1 = {_track["artist"]["id"]: _track["artist"]["name"] for _track in _user_recents_1}
_user_recent_artists_2 = {_track["artist"]["id"]: _track["artist"]["name"] for _track in _user_recents_2}

_user_recent_artists = list(_user_recent_artists_1.keys()) + list(_user_recent_artists_2.keys())

print("You listened recently about", len(_user_recent_artists), "artists!")

You listened recently about 15 artists!


## Top 3 for all Artists

In [9]:
_recommended_track_ids = []

for _artist_id in _user_recent_artists:

    _artist_top_3_tracks = _f_deezer_api("GET", "https://api.deezer.com/artist/{}/top?limit=3&access_token={}".format(_artist_id, _deezer_app_token))["data"]
    _artist_track_ids = {_track["id"]: _track["title"] for _track in _artist_top_3_tracks}

    _recommended_track_ids = _recommended_track_ids + [str(_track) for _track in set(_artist_track_ids.keys())]

_recommended_track_ids = list(set(_recommended_track_ids))
_formated_recommended_track_ids = ",".join(_recommended_track_ids).strip()

print("The algorithm recommended about", len(_recommended_track_ids), "new tracks!")

The algorithm recommended about 40 new tracks!


## Update RecSys tracks

### Drop Old Recommended Tracks from the RecSys playlist

In [10]:
_old_tracks = [str(_track["id"]) for _track in _f_deezer_api("GET", "https://api.deezer.com/playlist/{}?access_token={}".format(_deezer_recsys_playlist_id, _deezer_app_token))["tracks"]["data"]]
_formated_old_track_ids = ",".join(_old_tracks).strip()

_delete_tracks = _f_deezer_api("DELETE", "https://api.deezer.com/playlist/{}/tracks?access_token={}&songs={}".format(_deezer_recsys_playlist_id, _deezer_app_token, _formated_old_track_ids))

if _delete_tracks:
    print("The playlist was cleaned!")

The playlist was cleaned!


### Add the Newest Recommended Songs to RecSys playlist 

In [11]:
_updated_tracks = _f_deezer_api("POST", "https://api.deezer.com/playlist/{}/tracks?access_token={}&songs={}".format(_deezer_recsys_playlist_id, _deezer_app_token, _formated_recommended_track_ids))

if _updated_tracks:
    print("The playlist was updated with the new recommended songs!")

The playlist was updated with the new recommended songs!
